This is a benchmark for Autogluon Forecasting task.  
Assume that we have two COV19 dataset, train_data.csv and test_data.csv in the following format:  

|Date|ConfirmedCases|name|
| ------ | ------ | ------ |
|2020-01-22|0.0|Afghanistan_|
|2020-01-23|0.0|Afghanistan_|
|2020-01-24|0.0|Afghanistan_|
|2020-01-25|0.0|Afghanistan_|
|2020-01-26|0.0|Afghanistan_|

The comfirmedcases here is the cummulative comfirmed cases up to that date in a certain country.

In [1]:
# Load dataset
from autogluon.forecasting import ForecastingPredictor
from autogluon.forecasting import TabularDataset

train_data = TabularDataset("https://autogluon.s3-us-west-2.amazonaws.com/datasets/CovidTimeSeries/train.csv")
test_data = TabularDataset("https://autogluon.s3-us-west-2.amazonaws.com/datasets/CovidTimeSeries/test.csv")
prediction_length = 19
eval_metric = "mean_wQuantileLoss"

In [2]:
train_data.head()

,Date,ConfirmedCases,name
0,2020-01-22,0.0,Afghanistan_
1,2020-01-23,0.0,Afghanistan_
2,2020-01-24,0.0,Afghanistan_
3,2020-01-25,0.0,Afghanistan_
4,2020-01-26,0.0,Afghanistan_


## Directly using GluonTS to do forecasting

In [3]:
import pandas as pd


def rebuild_tabular(X, time_column, target_column, index_column=None):
    if index_column is None:
        X = X[[time_column, target_column]]
        X["index_column"] = ["time_series" for i in range(X.shape[0])]
        index_column = "index_column"
    time_list = sorted(list(set(X[time_column])))
    freq = pd.infer_freq(time_list)
    if freq is None:
        raise ValueError("Freq cannot be inferred. Check your dataset.")

    def reshape_dataframe(df):
        df = df.sort_values(by=index_column)
        data_dic = {index_column: sorted(list(set(df[index_column])))}

        for time in time_list:
            tmp = df[df[time_column] == time][[index_column, time_column, target_column]]
            tmp = tmp.pivot(index=index_column, columns=time_column, values=target_column)
            tmp_values = tmp[time].values
            data_dic[time] = tmp_values
        return pd.DataFrame(data_dic)

    X = reshape_dataframe(X)
    return X

In [4]:
rebuilt_train = rebuild_tabular(train_data, time_column="Date", target_column="ConfirmedCases", index_column="name")
rebuilt_test = rebuild_tabular(test_data, time_column="Date", target_column="ConfirmedCases", index_column="name")

In [5]:
rebuilt_train.head()

,name,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02
0,Afghanistan_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,4.0,5.0,7.0,7.0,7.0,11.0,16.0,21.0,22.0,22.0,22.0,24.0,24.0,40.0,40.0,74.0,84.0,94.0,110.0,110.0,120.0,170.0,174.0,237.0,273.0
1,Albania_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,10.0,12.0,23.0,33.0,38.0,42.0,51.0,55.0,59.0,64.0,70.0,76.0,89.0,104.0,123.0,146.0,174.0,186.0,197.0,212.0,223.0,243.0,259.0,277.0
2,Algeria_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,12.0,12.0,17.0,17.0,19.0,20.0,20.0,20.0,24.0,26.0,37.0,48.0,54.0,60.0,74.0,87.0,90.0,139.0,201.0,230.0,264.0,302.0,367.0,409.0,454.0,511.0,584.0,716.0,847.0,986.0
3,Andorra_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,39.0,39.0,53.0,75.0,88.0,113.0,133.0,164.0,188.0,224.0,267.0,308.0,334.0,370.0,376.0,390.0,428.0
4,Angola_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,3.0,3.0,3.0,4.0,4.0,5.0,7.0,7.0,7.0,8.0,8.0


In [6]:
rebuilt_test.head()

,name,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,...,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21
0,Afghanistan_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,7.0,11.0,16.0,21.0,22.0,22.0,22.0,24.0,24.0,40.0,40.0,74.0,84.0,94.0,110.0,110.0,120.0,170.0,174.0,237.0,273.0,281.0,299.0,349.0,367.0,423.0,444.0,484.0,521.0,555.0,607.0,665.0,714.0,784.0,840.0,906.0,933.0,996.0,1026.0,1092.0
1,Albania_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,33.0,38.0,42.0,51.0,55.0,59.0,64.0,70.0,76.0,89.0,104.0,123.0,146.0,174.0,186.0,197.0,212.0,223.0,243.0,259.0,277.0,304.0,333.0,361.0,377.0,383.0,400.0,409.0,416.0,433.0,446.0,467.0,475.0,494.0,518.0,539.0,548.0,562.0,584.0,609.0
2,Algeria_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,26.0,37.0,48.0,54.0,60.0,74.0,87.0,90.0,139.0,201.0,230.0,264.0,302.0,367.0,409.0,454.0,511.0,584.0,716.0,847.0,986.0,1171.0,1251.0,1320.0,1423.0,1468.0,1572.0,1666.0,1761.0,1825.0,1914.0,1983.0,2070.0,2160.0,2268.0,2418.0,2534.0,2629.0,2718.0,2811.0
3,Andorra_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,2.0,39.0,39.0,53.0,75.0,88.0,113.0,133.0,164.0,188.0,224.0,267.0,308.0,334.0,370.0,376.0,390.0,428.0,439.0,466.0,501.0,525.0,545.0,564.0,583.0,601.0,601.0,638.0,646.0,659.0,673.0,673.0,696.0,704.0,713.0,717.0,717.0
4,Angola_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,3.0,3.0,3.0,4.0,4.0,5.0,7.0,7.0,7.0,8.0,8.0,8.0,10.0,14.0,16.0,17.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,24.0,24.0,24.0,24.0


Train-Val Split

In [7]:
rebuilt_val = rebuilt_train.copy()
rebuilt_train = rebuilt_train.iloc[:, :-prediction_length]

Then transform the rebuilt tabular into GluonTS Listdata

In [8]:
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset

def create_gluonts_data(df, index_column):
    index = df[index_column]
    target = df.drop(index_column, axis=1)
    target_values = target.values
    date_list = target.columns
    freq = pd.infer_freq(date_list)
    data = [
        {
            FieldName.TARGET: target,
            FieldName.START: pd.Timestamp(date_list[0], freq=freq),
            FieldName.ITEM_ID: item_id
        }
        for (target, item_id) in zip(target_values, index)
    ]
    return ListDataset(data, freq)

In [9]:
gluonts_train_data = create_gluonts_data(rebuilt_train, "name")
gluonts_val_data = create_gluonts_data(rebuilt_val, "name")
gluonts_test_data = create_gluonts_data(rebuilt_test, "name")

Train and evaluate

In [11]:
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.mx.trainer import Trainer

params = {
    "prediction_length": 19,
    "freq": "D",
    "num_batches_per_epoch": 10,
    "epochs": 50,
    "quantiles": [0.1, 0.5, 0.9]
}
model = MQCNNEstimator.from_hyperparameters(**params)

predictor = model.train(gluonts_train_data)

  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 10/10 [00:01<00:00,  8.72it/s, epoch=50/50, avg_epoch_loss=15.9]


In [12]:
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from tqdm import tqdm


forecast_it, ts_it = make_evaluation_predictions(dataset=gluonts_test_data,
                                                 predictor=predictor,
                                                 num_samples=100)
forecasts, tss = list(forecast_it), list(ts_it)
# forecasts, tss = list(tqdm(forecast_it, total=len(gluonts_val_data))), list(tqdm(ts_it, total=len(gluonts_val_data)))
# print(forecasts[0], tss[0])
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(tss))
print(agg_metrics[eval_metric])

Running evaluation: 100%|██████████| 313/313 [00:00<00:00, 7082.37it/s]


0.13023926101671282


Get prediction results for 20 days after the dataset

In [13]:
result_dict = {}
predicted_targets = list(predictor.predict(gluonts_test_data))
index = sorted(list(set(train_data["name"])))

quantiles = [0.1, 0.5, 0.9]
for i in range(len(index)):
    tmp_dict = {}
    for quantile in quantiles:
        tmp_dict[quantile] = predicted_targets[i].quantile(str(quantile))
    df = pd.DataFrame(tmp_dict)
    df.index = pd.date_range(start=predicted_targets[i].start_date,
                             periods=prediction_length,
                             freq="D")
    result_dict[index[i]] = df
    
print(result_dict["Afghanistan_"])

                    0.1          0.5          0.9
2020-04-22  1045.543335  1301.114380  1291.978760
2020-04-23   911.084717  1325.646484  1581.920898
2020-04-24  1036.703369  1331.126831  1372.855225
2020-04-25   954.524963  1308.975952  1495.814087
2020-04-26  1045.396973  1343.073486  1451.277588
2020-04-27   973.158813  1362.873779  1756.688965
2020-04-28  1072.704468  1348.178101  1703.695557
2020-04-29  1066.819214  1337.218384  1564.016846
2020-04-30  1074.160522  1382.042847  1761.299316
2020-05-01  1012.753723  1339.383545  1796.303711
2020-05-02  1034.522705  1397.795654  1883.306519
2020-05-03  1028.629761  1440.088867  1805.396973
2020-05-04  1136.080688  1384.477783  1855.446899
2020-05-05  1074.078735  1399.549316  1893.104614
2020-05-06  1079.360962  1362.334961  1867.002930
2020-05-07  1133.655273  1357.222290  1950.235352
2020-05-08  1108.234863  1409.761841  1897.364136
2020-05-09  1136.624023  1353.572754  2108.404053
2020-05-10  1179.784546  1346.067627  2310.138184


## Using AutoGluon To do this forecasting

In [15]:
from autogluon.forecasting.task.forecasting.forecasting import Forecasting as task

import autogluon.core as ag

# change this to specify search strategy, can try bayesopt, random, or skopt
searcher_type = "random"
# change this to specify eval metric, one of ["MASE", "MAPE", "sMAPE", "mean_wQuantileLoss"]
eval_metric = "mean_wQuantileLoss"

path = "benchmark_models"
predictor = ForecastingPredictor(path=path, eval_metric=eval_metric).fit(train_data,
                                                                         prediction_length,
                                                                         index_column="name",
                                                                         target_column="ConfirmedCases",
                                                                         time_column="Date",
                                                                         hyperparameter_tune=True,
                                                                         quantiles=[0.1, 0.5, 0.9],
                                                                         refit_full=True,
                                                                         keep_only_best=True,
                                                                         set_best_to_refit_full=True,
                                                                         search_strategy=searcher_type,
                                                                         hyperparameters={
                                                                             "MQCNN": {
                                                                                 'context_length': ag.Int(70, 90,
                                                                                                          default=prediction_length * 4),
                                                                                 "num_batches_per_epoch": 10,
                                                                                 "epochs": 50},
                                                                         },
                                                                         num_trials=5

                                                                         )

/Users/yixiaxia/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Training with dataset in tabular format...
Finish rebuilding the data, showing the top five rows.
           name  2020-01-22  2020-01-23  ...  2020-03-31  2020-04-01  2020-04-02
0  Afghanistan_         0.0         0.0  ...       174.0       237.0       273.0
1      Albania_         0.0         0.0  ...       243.0       259.0       277.0
2      Algeria_         0.0         0.0  ...       716.0       847.0       986.0
3      Andorra_         0.0         0.0  ...       376.0       390.0       428.0
4       Angola_         0.0         0.0  ...         7.0         8.0         8.0

[5 rows x 73 columns]
Validation d

/Users/yixiaxia/Desktop/亚马逊工作/autogluon/core/src/autogluon/core/searcher/searcher.py:344: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  new_config = self.configspace.get_default_configuration().get_dictionary()
/Users/yixiaxia/Desktop/亚马逊工作/autogluon/core/src/autogluon/core/searcher/searcher.py:344: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


Epoch[0] Learning rate is 0.001

  0%|          | 0/10 [00:00<?, ?it/s]Number of parameters in ForkingSeq2SeqTrainingNetwork: 57784
Start model training


learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


...


100%|██████████| 10/10 [00:02<00:00,  3.83it/s, epoch=46/50, avg_epoch_loss=17.1]
Epoch[45] Elapsed time 2.624 seconds
Epoch[45] Evaluation metric 'epoch_loss'=17.057621
Epoch[46] Learning rate is 0.001

100%|██████████| 10/10 [00:03<00:00,  3.11it/s, epoch=39/50, avg_epoch_loss=19]
Epoch[38] Elapsed time 3.223 seconds
Epoch[38] Evaluation metric 'epoch_loss'=18.991143
Epoch[39] Learning rate is 0.001

100%|██████████| 10/10 [00:03<00:00,  3.20it/s, epoch=39/50, avg_epoch_loss=18.7]
Epoch[38] Elapsed time 3.139 seconds
Epoch[38] Evaluation metric 'epoch_loss'=18.745980
Epoch[39] Learning rate is 0.001

100%|██████████| 10/10 [00:03<00:00,  3.04it/s, epoch=37/50, avg_epoch_loss=20.3]
Epoch[36] Elapsed time 3.293 seconds
Epoch[36] Evaluation metric 'epoch_loss'=20.291594
Epoch[37] Learning rate is 0.001
100%|██████████| 10/10 [00:02<00:00,  3.66it/s, epoch=46/50, avg_epoch_loss=17.2]
Epoch[45] Elapsed time 2.746 seconds
Epoch[45] Evaluation metric 'epoch_loss'=17.222621
Epoch[46] Learni


100%|██████████| 10/10 [00:02<00:00,  4.43it/s, epoch=43/50, avg_epoch_loss=17.9]
Epoch[42] Elapsed time 2.264 seconds
Epoch[42] Evaluation metric 'epoch_loss'=17.911473
Epoch[43] Learning rate is 0.001
100%|██████████| 10/10 [00:02<00:00,  4.60it/s, epoch=46/50, avg_epoch_loss=17.1]
Epoch[45] Elapsed time 2.185 seconds
Epoch[45] Evaluation metric 'epoch_loss'=17.102557
Epoch[46] Learning rate is 0.001

100%|██████████| 10/10 [00:02<00:00,  4.44it/s, epoch=46/50, avg_epoch_loss=17.2]
Epoch[45] Elapsed time 2.262 seconds
Epoch[45] Evaluation metric 'epoch_loss'=17.221098
Epoch[46] Learning rate is 0.001

100%|██████████| 10/10 [00:02<00:00,  3.99it/s, epoch=44/50, avg_epoch_loss=16.5]
Epoch[43] Elapsed time 2.511 seconds
Epoch[43] Evaluation metric 'epoch_loss'=16.520939
Epoch[44] Learning rate is 0.001
100%|██████████| 10/10 [00:02<00:00,  4.01it/s, epoch=47/50, avg_epoch_loss=17.2]
Epoch[46] Elapsed time 2.501 seconds
Epoch[46] Evaluation metric 'epoch_loss'=17.240703
Epoch[47] Learn

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


Number of parameters in ForkingSeq2SeqTrainingNetwork: 57784
100%|██████████| 10/10 [00:01<00:00,  6.24it/s, epoch=1/50, avg_epoch_loss=132]
Epoch[0] Elapsed time 1.604 seconds
Epoch[0] Evaluation metric 'epoch_loss'=131.853205
Epoch[1] Learning rate is 0.001
100%|██████████| 10/10 [00:01<00:00,  6.78it/s, epoch=2/50, avg_epoch_loss=130]
Epoch[1] Elapsed time 1.476 seconds
Epoch[1] Evaluation metric 'epoch_loss'=129.974428
Epoch[2] Learning rate is 0.001
100%|██████████| 10/10 [00:01<00:00,  6.58it/s, epoch=3/50, avg_epoch_loss=128]
Epoch[2] Elapsed time 1.521 seconds
Epoch[2] Evaluation metric 'epoch_loss'=127.726345
Epoch[3] Learning rate is 0.001
100%|██████████| 10/10 [00:01<00:00,  6.58it/s, epoch=4/50, avg_epoch_loss=122]
Epoch[3] Elapsed time 1.522 seconds
Epoch[3] Evaluation metric 'epoch_loss'=121.578448
Epoch[4] Learning rate is 0.001
100%|██████████| 10/10 [00:01<00:00,  6.69it/s, epoch=5/50, avg_epoch_loss=107]
Epoch[4] Elapsed time 1.496 seconds
Epoch[4] Evaluation metric 

100%|██████████| 10/10 [00:01<00:00,  6.55it/s, epoch=41/50, avg_epoch_loss=49.3]
Epoch[40] Elapsed time 1.528 seconds
Epoch[40] Evaluation metric 'epoch_loss'=49.300569
Epoch[41] Learning rate is 0.001
100%|██████████| 10/10 [00:01<00:00,  6.66it/s, epoch=42/50, avg_epoch_loss=46.9]
Epoch[41] Elapsed time 1.504 seconds
Epoch[41] Evaluation metric 'epoch_loss'=46.879207
Epoch[42] Learning rate is 0.001
100%|██████████| 10/10 [00:01<00:00,  6.47it/s, epoch=43/50, avg_epoch_loss=49.5]
Epoch[42] Elapsed time 1.546 seconds
Epoch[42] Evaluation metric 'epoch_loss'=49.545385
Epoch[43] Learning rate is 0.001
100%|██████████| 10/10 [00:01<00:00,  6.46it/s, epoch=44/50, avg_epoch_loss=46.3]
Epoch[43] Elapsed time 1.551 seconds
Epoch[43] Evaluation metric 'epoch_loss'=46.286301
Epoch[44] Learning rate is 0.001
100%|██████████| 10/10 [00:01<00:00,  6.22it/s, epoch=45/50, avg_epoch_loss=45.6]
Epoch[44] Elapsed time 1.609 seconds
Epoch[44] Evaluation metric 'epoch_loss'=45.585882
Epoch[45] Learning

Evaluate the model, and 

In [16]:
print(predictor.evaluate(test_data))

/Users/yixiaxia/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Does not specify model, will by default use the model with the best validation score for evaluation
Running evaluation: 100%|██████████| 313/313 [00:00<00:00, 5163.73it/s]


0.1279880999040174


Get prediction results for 20 days after the dataset

In [17]:
predictions = predictor.predict(test_data, quantiles=[0.1, 0.5, 0.9])
print(predictions['Afghanistan_'])

Does not specify model, will by default use the model with the best validation score for prediction
Predicting with model MQCNN/trial_9


                    0.1          0.5          0.9
2020-04-22  1052.921753  1143.598877  1295.557739
2020-04-23   884.967163  1337.154297  1575.992432
2020-04-24  1068.580444  1326.074341  1396.617798
2020-04-25   970.084717  1333.021851  1509.136230
2020-04-26  1050.712036  1294.856689  1457.340210
2020-04-27   936.677429  1398.337769  1735.085815
2020-04-28  1141.875488  1369.357666  1659.464478
2020-04-29  1042.039307  1352.041748  1608.570557
2020-04-30  1121.011353  1380.281006  1720.915283
2020-05-01   998.869995  1343.244995  1802.393188
2020-05-02  1060.995483  1388.346680  1860.138916
2020-05-03  1159.115356  1373.995239  1780.534668
2020-05-04  1117.194336  1376.017944  1861.159058
2020-05-05  1063.938721  1427.076416  1890.203125
2020-05-06  1122.769165  1376.724731  1834.982788
2020-05-07  1145.380371  1371.960693  1920.892212
2020-05-08  1140.659790  1394.517456  1891.408936
2020-05-09  1108.873169  1362.765869  2065.429688
2020-05-10  1174.494019  1329.412231  2263.693359
